# Učitavanje spremljenog modela

In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import torch
import pandas as pd
from datasets import Dataset

/Users/patricijamarijanovic/ZAVRSNI RAD/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Učitavanje modela i tokenizatora

In [2]:
model_path = "./spremljeni_model"
model = AutoModelForSequenceClassification.from_pretrained(model_path) # ucitavanje modela
tokenizer = AutoTokenizer.from_pretrained(model_path) # ucitavanje tokenizatora

Ovo nije prebitno, ali moram staviti da funkcionira:

In [3]:
# Dummy training args – neće se trenirati, samo trebaju Traineru
training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=1,
    do_train=False,
    do_eval=False,
    logging_dir="./logs"
)

# Inicijalizacija trainera (iako ga koristimo samo za predikciju)
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args
)

/var/folders/mk/drz6rhr972v4l3bn903sdl600000gn/T/ipykernel_12223/443292532.py:11: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Preprocessing rečenice

In [6]:
# Prilagodi preprocess function za samo predikciju (bez labela)

def preprocess_function(df):

    # tokenizacija stupca "text"
    tokenized_inputs = tokenizer( # koristi se distilBERT tokenizer
        df['text'], # sto treba tokenizirati
        padding='longest', # dodaje 0 gdje treba da svi inputi budu iste duljine
        truncation=True, # reže preduge rečenice (>512 tokena)
        max_length=512, # osigurava da ništa ne prelazi 512 tokena (maks. za BERT)
        return_tensors='pt' # vraća podatke kao PyTorch tenzore (torch.Tensor) umjesto Python lista
    ) # vraca rjecnik s kljucevima 'input_ids', 'attention_mask'

    return tokenized_inputs

In [5]:
labels = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
id2label = {i: label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}

Predikcija iz rečenice

In [9]:
# funkcija za testiranje pojedinacne recenice
def predikcija_iz_recenice(my_sentence):
    dummy_emotion = 0  # mora biti nešto iz label2id
    # Pandas DataFrame
    df = pd.DataFrame({
        "text": [my_sentence],
        "label": [dummy_emotion]
    })

    # HuggingFace Dataset
    sentence_dataset = Dataset.from_pandas(df)

    # Koristi mapiranje samo za tekst
    sentence_dataset = sentence_dataset.map(lambda x: preprocess_function(x), batched=True)

    # Predikcija
    prediction = trainer.predict(sentence_dataset)

    # Uzmi klasu s najvećim scoreom
    predicted_class = prediction.predictions.argmax(axis=1)[0]

    # Ako imaš id2label mapu
    predicted_label = id2label[predicted_class]
    print(f"Predikcija emocije: {predicted_label}")

In [14]:
predikcija_iz_recenice("my cat died today")

Map: 100%|██████████| 1/1 [00:00<00:00, 252.27 examples/s]


Predikcija emocije: sadness
